In [1]:
#Test with test data
import matplotlib.pyplot as plt
import os
import random

import tensorflow as tf
import cv2
import numpy as np

from segmentation_models.metrics import iou_score
from segmentation_models import Unet
import segmentation_models as sm


sm.set_framework("tf.keras")
sm.framework()

#Initialize parameters
BACKBONE = "resnet34"
preprocess_input = sm.get_preprocessing(BACKBONE)
batch_size = 16
w, h = 384, 384

#initialize model
opt=tf.keras.optimizers.Adam(0.00001)
model= Unet(BACKBONE,encoder_weights = "imagenet",classes = 3,activation="softmax",input_shape = (384, 384 ,3),encoder_freeze = False)
loss1 = sm.losses.categorical_focal_dice_loss

model.compile(optimizer=opt,loss=loss1,metrics=[iou_score])

#Test with val data
model.load_weights("checkpoint_1.hdf5")

Segmentation Models: using `keras` framework.


In [2]:
'''
from keras.models import load_model 
model = load_model('best3.h5', compile = False)
'''

"\nfrom keras.models import load_model \nmodel = load_model('best3.h5', compile = False)\n"

In [3]:
import os

path ="test/test"
#we shall store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))

In [4]:
import os
import cv2

test_folder = path
save_folder = "predict_mask5"

for file in files:
#for i in range(1):

    image_path = os.path.join(root, file)
    save_path = os.path.join(save_folder, file)
    save_path = save_path


    # Read input image
    image1 = cv2.imread(image_path)
    w, h = image1.shape[0], image1.shape[1]
    image = cv2.resize(image1, (384, 384))


    mask_predict = model.predict(image[np.newaxis, :, :, :])

    
    z = mask_predict[0]#[:, :, 0]
    z = cv2.resize(z, (h, w))

    tmp = np.zeros_like(image1)
    tmp[:, :, 0] = 0
    tmp[:, :, 1] = np.where(z[:, :, 1] * 255 > 240, 255, 0)
    tmp[:, :, 2] = np.where(z[:, :, 2] * 255 > 240, 255, 0)

    
    #cv2.imshow('a', tmp)
    cv2.imwrite(save_path[:-4] + "png", tmp)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    '''
    plt.imshow(tmp)
    print(tmp.shape)
    plt.show()
    '''
    

1/1 [==============================] - 0s 16ms/step


In [5]:
import numpy as np
import pandas as pd
import cv2
import os

def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = 'predict_mask5' # change this to the path to your output mask folder
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output(8).csv', index=False)

predict_mask5\019410b1fcf0625f608b4ce97629ab55.png
predict_mask5\02fa602bb3c7abacdbd7e6afd56ea7bc.png
predict_mask5\0398846f67b5df7cdf3f33c3ca4d5060.png
predict_mask5\05734fbeedd0f9da760db74a29abdb04.png
predict_mask5\05b78a91391adc0bb223c4eaf3372eae.png
predict_mask5\0619ebebe9e9c9d00a4262b4fe4a5a95.png
predict_mask5\0626ab4ec3d46e602b296cc5cfd263f1.png
predict_mask5\0a0317371a966bf4b3466463a3c64db1.png
predict_mask5\0a5f3601ad4f13ccf1f4b331a412fc44.png
predict_mask5\0af3feff05dec1eb3a70b145a7d8d3b6.png
predict_mask5\0fca6a4248a41e8db8b4ed633b456aaa.png
predict_mask5\1002ec4a1fe748f3085f1ce88cbdf366.png
predict_mask5\1209db6dcdda5cc8a788edaeb6aa460a.png
predict_mask5\13dd311a65d2b46d0a6085835c525af6.png
predict_mask5\1531871f2fd85a04faeeb2b535797395.png
predict_mask5\15fc656702fa602bb3c7abacdbd7e6af.png
predict_mask5\1ad4f13ccf1f4b331a412fc44655fb51.png
predict_mask5\1b62f15ec83b97bb11e8e0c4416c1931.png
predict_mask5\1c0e9082ea2c193ac8d551c149b60f29.png
predict_mask5\1db239dda50f954ba